In [1]:
pip install pulp

Note: you may need to restart the kernel to use updated packages.


In [5]:
# -*- coding: utf-8 -*-
# -*- coding: utf-8 -*-
"""
Created on Tue Aug  3 16:29:02 2021

@author: hhzhe
"""
# -*- coding: utf-8 -*-


from pulp import *

lst = []
with open('cgainput.csv') as f:
    lines = f.readlines()
    for line in lines:
        z = line.split(',')
        z[-1] = z[-1].strip()
        lst.append(z)
        
f.close()
#print(lst)

gpr = float(lst[0][1])
print(gpr)
ref_dct = {}

sitearea = float(lst[1][1])

#create list from default list
indexlst = []
for i in range(3, len(lst)):
    indexlst.append(lst[i][0])
    
#print(indexlst)


arealst = []
for i in range(3, len(lst)):
    arealst.append(float(lst[i][1]))
#print(arealst)

minlst = []
maxlst = []
for i in range(3, len(lst)):
        if lst[i][2] == "NA" and lst[i][3] != "NA":
            minlst.append("NA")
            maxlst.append(int(lst[i][3]))
        elif lst[i][2] != "NA" and lst[i][3] == "NA":
            minlst.append(int(lst[i][2]))
            maxlst.append("NA")
        elif lst[i][2] == "NA" and lst[i][3] == "NA":
            minlst.append(0)
            maxlst.append("NA")
        else:
            minlst.append(int(lst[i][2]))
            maxlst.append(int(lst[i][3]))

# create ref dictionary
ref_dct = {}
for i in range(len(indexlst)):
    ref_dct[indexlst[i]] = {"AREA":arealst[i],"MIN":minlst[i],"MAX":maxlst[i],"F1":0,"F2":0,"F3":0,"F4":0}
print("Original:")
#print(ref_dct)

# Create variables
h1_var = LpVariable.dicts("h", indexlst,lowBound = 0, cat = LpInteger)
h2_var = LpVariable.dicts("h", indexlst,lowBound = 0, cat = LpInteger)
h3_var = LpVariable.dicts("h", indexlst,lowBound = 0, cat = LpInteger)
h4_var = LpVariable.dicts("h", indexlst,lowBound = 0, cat = LpInteger)

# Objective function
prob1 = LpProblem("Test1", LpMinimize) # = but min building
prob2 = LpProblem("Test2", LpMaximize) # = but max building
prob3 = LpProblem("Test3", LpMinimize) # >= GPR
prob4 = LpProblem("Test4", LpMaximize) # <= GPR
#objective function is added first
prob1 += lpSum([h1_var[x] for x in indexlst]), "Minimize heights of buildings"
prob2 += lpSum([h2_var[x] for x in indexlst]), "Maximize heights of buildings"
prob3 += lpSum([h3_var[x] for x in indexlst]), "Minimize heights of buildings"
prob4 += lpSum([h4_var[x] for x in indexlst]), "Maximize heights of buildings"
#GPR constraint
prob1 += lpSum([ref_dct[x]["AREA"]*h1_var[x] for x in indexlst]) == sitearea*gpr
prob2 += lpSum([ref_dct[x]["AREA"]*h2_var[x] for x in indexlst]) == sitearea*gpr
prob3 += lpSum([ref_dct[x]["AREA"]*h3_var[x] for x in indexlst]) >= sitearea*gpr
prob4 += lpSum([ref_dct[x]["AREA"]*h4_var[x] for x in indexlst]) <= sitearea*gpr
# add height constraints
for x in indexlst:
    if ref_dct[x]["MIN"] == "NA" and ref_dct[x]["MAX"] != "NA":
        prob1 += h1_var[x] <= ref_dct[x]["MAX"]
        prob2 += h2_var[x] <= ref_dct[x]["MAX"]
        prob3 += h3_var[x] <= ref_dct[x]["MAX"]
        prob4 += h4_var[x] <= ref_dct[x]["MAX"]
    elif ref_dct[x]["MAX"] == "NA" and ref_dct[x]["MIN"] != "NA":
        prob1 += h1_var[x] >= ref_dct[x]["MIN"]
        prob2 += h2_var[x] >= ref_dct[x]["MIN"]
        prob3 += h3_var[x] >= ref_dct[x]["MIN"]
        prob4 += h4_var[x] >= ref_dct[x]["MIN"]
    else:
        prob1 += h1_var[x] >= ref_dct[x]["MIN"]
        prob1 += h1_var[x] <= ref_dct[x]["MAX"]
        prob2 += h2_var[x] >= ref_dct[x]["MIN"]
        prob2 += h2_var[x] <= ref_dct[x]["MAX"]
        prob3 += h3_var[x] >= ref_dct[x]["MIN"]
        prob3 += h3_var[x] <= ref_dct[x]["MAX"]
        prob4 += h4_var[x] >= ref_dct[x]["MIN"]
        prob4 += h4_var[x] <= ref_dct[x]["MAX"]
print("problem solver")
prob1.solve()
prob2.solve()
prob3.solve()
prob4.solve()
print("Status1:", LpStatus[prob1.status])
print("Status2:", LpStatus[prob2.status])
print("Status3:", LpStatus[prob3.status])
print("Status4:", LpStatus[prob4.status])
# new heights
for x in indexlst:
    ref_dct[x]["F1"] = int(h1_var[x].varValue)
    ref_dct[x]["F2"] = int(h2_var[x].varValue)
    ref_dct[x]["F3"] = int(h3_var[x].varValue)
    ref_dct[x]["F4"] = int(h4_var[x].varValue)
    
print("Final:")
#print(ref_dct)


# calculate new gpr:
gprlst = []
temp1 = temp2 = temp3 = temp4 = 0
for x in indexlst:
    temp1 += ref_dct[x]["F1"]*ref_dct[x]["AREA"]
    temp2 += ref_dct[x]["F2"]*ref_dct[x]["AREA"]
    temp3 += ref_dct[x]["F2"]*ref_dct[x]["AREA"]
    temp4 += ref_dct[x]["F3"]*ref_dct[x]["AREA"]
    
GPR1 = round(temp1/sitearea,4)
GPR2 = round(temp2/sitearea,4)
GPR3 = round(temp3/sitearea,4)
GPR4 = round(temp4/sitearea,4)
print(GPR1,GPR2,GPR3,GPR4)

# Write to file
f2 = open("output.csv","w")
output = []
gprline = "GPR,"+str(gpr)+",,,,,,\n"
output.append(gprline)
output.append("SITEAREA,{0}\n".format(sitearea))
output.append("INDEX,AREA,MIN,MAX,V1,V2,V3,V4\n")
for x in ref_dct:
    output.append("{0},{1},{2},{3},{4},{5},{6},{7}\n".format(x,ref_dct[x]["AREA"],ref_dct[x]["MIN"],ref_dct[x]["MAX"],ref_dct[x]["F1"],ref_dct[x]["F2"],ref_dct[x]["F3"],ref_dct[x]["F4"]))
# print(output)
output.append("CalcGPR,,,,{0},{1},{2},{3}".format(GPR1,GPR2,GPR3,GPR4))
f2.writelines(output)
f2.close()
print("Write to file")

4.2
Original:
problem solver
Status1: Optimal
Status2: Optimal
Status3: Optimal
Status4: Optimal
Final:
4.2 4.2 4.2 4.2665
